In [1]:
import pandas as pd

ts_data = pd.read_parquet('../data/transformed/ts_data_2025_01.parquet')
ts_data

,pickup_hour,rides,pickup_location_id
0,2025-01-01 00:00:00,28,4
1,2025-01-01 01:00:00,54,4
2,2025-01-01 02:00:00,52,4
3,2025-01-01 03:00:00,42,4
4,2025-01-01 04:00:00,32,4
...,...,...,...
194179,2025-01-31 19:00:00,0,187
194180,2025-01-31 20:00:00,0,187
194181,2025-01-31 21:00:00,0,187
194182,2025-01-31 22:00:00,0,187


In [2]:
ts_data_one_location = ts_data.loc[ts_data.pickup_location_id == 43, :].reset_index(drop=True)
ts_data_one_location.head(15)

,pickup_hour,rides,pickup_location_id
0,2025-01-01 00:00:00,69,43
1,2025-01-01 01:00:00,56,43
2,2025-01-01 02:00:00,20,43
3,2025-01-01 03:00:00,10,43
4,2025-01-01 04:00:00,9,43
5,2025-01-01 05:00:00,1,43
6,2025-01-01 06:00:00,4,43
7,2025-01-01 07:00:00,9,43
8,2025-01-01 08:00:00,9,43
9,2025-01-01 09:00:00,25,43


In [3]:
def get_cutoff_indices(
    data: pd.DataFrame,
    n_features: int,
    step_size: int
    ) -> list:

        stop_position = len(data) - 1
        
        # Start the first sub-sequence at index position 0
        subseq_first_idx = 0
        subseq_mid_idx = n_features
        subseq_last_idx = n_features + 1
        indices = []
        
        while subseq_last_idx <= stop_position:
            indices.append((subseq_first_idx, subseq_mid_idx, subseq_last_idx))
            
            subseq_first_idx += step_size
            subseq_mid_idx += step_size
            subseq_last_idx += step_size

        return indices

In [4]:
n_features = 24
step_size = 1

indices = get_cutoff_indices(
    ts_data_one_location,
    n_features,
    step_size
)
indices[:5]

[(0, 24, 25), (1, 25, 26), (2, 26, 27), (3, 27, 28), (4, 28, 29)]

In [7]:
import numpy as np

n_examples = len(indices)
x = np.ndarray(shape=(n_examples, n_features), dtype=np.float32)
y = np.ndarray(shape=(n_examples), dtype=np.float32)
pickup_hours = []

for i, idx in enumerate(indices):
    x[i, :] = ts_data_one_location.iloc[idx[0]:idx[1]]['rides'].values
    #y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['rides'].values
    y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['rides'].values.item()
    pickup_hours.append(ts_data_one_location.iloc[idx[1]]['pickup_hour'])

In [11]:
print(f'{x.shape=}')
print(f'{x=}')
print(f'{pickup_hours[:5]=}')

x.shape=(719, 24)
x=array([[ 69.,  56.,  20., ...,  34.,  25.,   6.],
       [ 56.,  20.,  10., ...,  25.,   6.,   5.],
       [ 20.,  10.,   9., ...,   6.,   5.,   1.],
       ...,
       [ 61.,  68.,  64., ..., 206., 123., 112.],
       [ 68.,  64.,  20., ..., 123., 112., 101.],
       [ 64.,  20.,   5., ..., 112., 101.,  71.]],
      shape=(719, 24), dtype=float32)
pickup_hours[:5]=[Timestamp('2025-01-02 00:00:00'), Timestamp('2025-01-02 01:00:00'), Timestamp('2025-01-02 02:00:00'), Timestamp('2025-01-02 03:00:00'), Timestamp('2025-01-02 04:00:00')]


In [12]:
features_one_location = pd.DataFrame(
    x,
    columns=[f'rides_previous_{i+1}_hour' for i in reversed(range(n_features))]
)
features_one_location

,rides_previous_24_hour,rides_previous_23_hour,rides_previous_22_hour,rides_previous_21_hour,rides_previous_20_hour,rides_previous_19_hour,rides_previous_18_hour,rides_previous_17_hour,rides_previous_16_hour,rides_previous_15_hour,...,rides_previous_10_hour,rides_previous_9_hour,rides_previous_8_hour,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour
0,69.0,56.0,20.0,10.0,9.0,1.0,4.0,9.0,9.0,25.0,...,126.0,108.0,109.0,70.0,48.0,53.0,25.0,34.0,25.0,6.0
1,56.0,20.0,10.0,9.0,1.0,4.0,9.0,9.0,25.0,53.0,...,108.0,109.0,70.0,48.0,53.0,25.0,34.0,25.0,6.0,5.0
2,20.0,10.0,9.0,1.0,4.0,9.0,9.0,25.0,53.0,88.0,...,109.0,70.0,48.0,53.0,25.0,34.0,25.0,6.0,5.0,1.0
3,10.0,9.0,1.0,4.0,9.0,9.0,25.0,53.0,88.0,92.0,...,70.0,48.0,53.0,25.0,34.0,25.0,6.0,5.0,1.0,0.0
4,9.0,1.0,4.0,9.0,9.0,25.0,53.0,88.0,92.0,134.0,...,48.0,53.0,25.0,34.0,25.0,6.0,5.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,88.0,59.0,61.0,68.0,64.0,20.0,5.0,4.0,1.0,1.0,...,81.0,69.0,76.0,78.0,108.0,98.0,145.0,160.0,142.0,206.0
715,59.0,61.0,68.0,64.0,20.0,5.0,4.0,1.0,1.0,1.0,...,69.0,76.0,78.0,108.0,98.0,145.0,160.0,142.0,206.0,123.0
716,61.0,68.0,64.0,20.0,5.0,4.0,1.0,1.0,1.0,4.0,...,76.0,78.0,108.0,98.0,145.0,160.0,142.0,206.0,123.0,112.0
717,68.0,64.0,20.0,5.0,4.0,1.0,1.0,1.0,4.0,6.0,...,78.0,108.0,98.0,145.0,160.0,142.0,206.0,123.0,112.0,101.0


In [13]:
targets_one_location = pd.DataFrame(y, columns=[f'target_rides_next_hour'])
targets_one_location

,target_rides_next_hour
0,5.0
1,1.0
2,0.0
3,0.0
4,1.0
...,...
714,123.0
715,112.0
716,101.0
717,71.0


In [17]:
from tqdm import tqdm

def transform_ts_data_into_features_and_target(
    ts_data: pd.DataFrame,
    input_seq_len: int,
    step_size: int
) -> pd.DataFrame:
    """
    Slices and transposes data from time-series format into a (features, target)
    format that we can use to train Supervised ML models
    """
    assert set(ts_data.columns) == {'pickup_hour', 'rides', 'pickup_location_id'}

    location_ids = ts_data['pickup_location_id'].unique()
    features = pd.DataFrame()
    targets = pd.DataFrame()
    
    for location_id in tqdm(location_ids):
        
        # keep only ts data for this `location_id`
        ts_data_one_location = ts_data.loc[
            ts_data.pickup_location_id == location_id, 
            ['pickup_hour', 'rides']
        ]

        # pre-compute cutoff indices to split dataframe rows
        indices = get_cutoff_indices(
            ts_data_one_location,
            input_seq_len,
            step_size
        )

        # slice and transpose data into numpy arrays for features and targets
        n_examples = len(indices)
        x = np.ndarray(shape=(n_examples, input_seq_len), dtype=np.float32)
        y = np.ndarray(shape=(n_examples), dtype=np.float32)
        pickup_hours = []
        for i, idx in enumerate(indices):
            x[i, :] = ts_data_one_location.iloc[idx[0]:idx[1]]['rides'].values
            y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['rides'].values.item()
            pickup_hours.append(ts_data_one_location.iloc[idx[1]]['pickup_hour'])

        # numpy -> pandas
        features_one_location = pd.DataFrame(
            x,
            columns=[f'rides_previous_{i+1}_hour' for i in reversed(range(input_seq_len))]
        )
        features_one_location['pickup_hour'] = pickup_hours
        features_one_location['pickup_location_id'] = location_id

        # numpy -> pandas
        targets_one_location = pd.DataFrame(y, columns=[f'target_rides_next_hour'])

        # concatenate results
        features = pd.concat([features, features_one_location])
        targets = pd.concat([targets, targets_one_location])

    features.reset_index(inplace=True, drop=True)
    targets.reset_index(inplace=True, drop=True)

    return features, targets['target_rides_next_hour']

In [18]:
features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*7*1, # one week of history
    step_size=24,
)

print(f'{features.shape=}')
print(f'{targets.shape=}')

100%|██████████| 261/261 [00:00<00:00, 521.83it/s]

features.shape=(6264, 170)
targets.shape=(6264,)


In [19]:
features

,rides_previous_168_hour,rides_previous_167_hour,rides_previous_166_hour,rides_previous_165_hour,rides_previous_164_hour,rides_previous_163_hour,rides_previous_162_hour,rides_previous_161_hour,rides_previous_160_hour,rides_previous_159_hour,...,rides_previous_8_hour,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id
0,28.0,54.0,52.0,42.0,32.0,5.0,5.0,3.0,1.0,2.0,...,3.0,4.0,1.0,3.0,0.0,4.0,2.0,0.0,2025-01-08,4
1,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,4.0,4.0,...,2.0,4.0,10.0,2.0,2.0,2.0,7.0,1.0,2025-01-09,4
2,1.0,5.0,6.0,1.0,1.0,1.0,0.0,2.0,9.0,4.0,...,2.0,2.0,9.0,6.0,8.0,5.0,14.0,17.0,2025-01-10,4
3,20.0,31.0,21.0,13.0,2.0,0.0,1.0,0.0,1.0,5.0,...,3.0,4.0,18.0,9.0,5.0,9.0,16.0,44.0,2025-01-11,4
4,22.0,46.0,34.0,13.0,4.0,0.0,0.0,1.0,2.0,2.0,...,6.0,4.0,9.0,16.0,15.0,16.0,35.0,67.0,2025-01-12,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-01-27,187
6260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-01-28,187
6261,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-01-29,187
6262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-01-30,187
